In [66]:
import pandas as pd
import numpy as np
import re
import os

df = pd.read_json('../Data/Serie_A_2017_games.json')

df_squad = pd.read_json('../Data/Serie_A_2018_squads.json').T

df_transposto = df.transpose()

df_transposto.head(29)

,Home,Away,Result,Players,Goals,Changes,Yellow cards,Red cards
1,Fluminense / RJ,Santos / SP,3 X 2,[[12Diego Cava ... Diego Cavalieri T(g)P137990...,[03:00 1T9NRJose Henrique da Silva Dourado Flu...,[17:00 2TFluminense/RJ 5 - Lucas Pierre Santos...,"[09:00 2T9Ricardo de Oliveira Santos/SP, 16:00...",[]
2,Flamengo / RJ,Atlético Mineiro / MG,1 X 1,[[38Alex Muralha Alex Roberto Santana ... T(g)...,[23:00 1T42NRMatheus Gonçalves Savio Flamengo/...,[45:00 INTAtlético/MG 10 - Juan Ramon Cazares ...,"[14:00 2T15Ralph Machado Dias Atlético/MG, 20:...",[]
3,Palmeiras / SP,Vasco da Gama / RJ,4 X 0,[[1Fernando P ... Fernando Buttenbende ... T(g...,[40:00 1T18NRAlejandro Abraham Guerra Morales ...,[12:00 2TVasco da Gama/RJ 19 - Paulo Vitor Fer...,[32:00 2T3Jomar Herculano Lourenco Vasco da Ga...,[]
4,Corinthians / SP,Chapecoense / SC,1 X 1,"[[12Cassio Cassio Ramos T(g)P166276, Corinthia...",[21:00 1T7NRJoao Alves de Assis Silva Corinthi...,[43:00 1TCorinthians/SP 34 - Pedro Henrique Ri...,[17:00 2T23Fagner Conserva Lemos Corinthians/S...,[]
5,Cruzeiro / MG,São Paulo / SP,1 X 0,"[[1Fabio Fabio Deivson Lopes ... T(g)P129292,...",[02:00 2T9NRRamon Dario Ábila Cruzeiro/MG],[45:00 INTSão Paulo/SP 31 - Luiz de Araujo Gui...,[+01:00 2T3Rodrigo Caio Coquette Russo São Pau...,[]
6,Coritiba / PR,Atlético / GO,4 X 1,[[84Wilson Wilson Rodrigues de ... T(g)P15550...,[31:00 1T91NRHenrique Almeida Caixeta Nascente...,[18:00 1TCoritiba/PR 26 - Tomas Almino Bastos ...,"[41:00 1T5Alan Santos da Silva Coritiba/PR, 24...",[]
7,Grêmio / RS,Botafogo / RJ,2 X 0,"[[1M. Grohe Marcelo Grohe T(g)P168351, Grêmio ...","[45:00 1T17NRRamiro Moschen Benetti Grêmio/RS,...",[12:00 2TBotafogo/RJ 20 - Guilherme Augusto Vi...,[14:00 2T8Bruno Cesar Pereira Silva Botafogo/R...,[]
8,Bahia / BA,Athletico Paranaense / PR,6 X 2,"[[1Jean Jean Paulo Fernandes ... T(g)P420409, ...",[14:00 1T17NRGuilherme Milhomem Gusmao Atlétic...,[09:00 2TAtlético/PR 20 - Matheus Rossetto 33 ...,[23:00 2T45Marcos do Nascimento Teixeira Atlét...,[28:00 2T45Marcos do Nascimento Teixeira - Atl...
9,Ponte Preta / SP,Sport / PE,4 X 0,[[1Aranha Mario Lucio Duarte Costa T(g)P142720...,[39:00 1T9NRLucca Borges de Brito Ponte Preta/...,[45:00 INTSport/PE 37 - Edimar Ribeiro da Cost...,[41:00 1T5Rodrigo Vasconcelos Oliveira Sport/P...,[]
10,Avaí / SC,Vitória / BA,0 X 0,"[[1Mauricio Mauricio Kozlinski T(g)P296645, Av...",[],[22:00 2TAvaí/SC 17 - Welington Simiao 10 - Ma...,[29:00 2T21Rene Ferreira dos Santos Vitória/BA...,[]


In [67]:
# Selecionar linhas onde 'Palmeiras / SP' aparece em 'Home' ou 'Away'
#df_transposto.loc[(df_transposto['Home'] == 'Palmeiras / SP') | (df_transposto['Away'] == 'Palmeiras / SP')]


In [68]:
df_squad.head(9)

,0,1,2,3,4,5,6,7,8
1,"{'Home': {'Squad': ['129292', '174360', '53542...","{'Home': {'Squad': ['129292', '174360', '53542...","{'Home': {'Squad': ['129292', '174360', '53542...","{'Home': {'Squad': ['129292', '174360', '15728...","{'Home': {'Squad': ['129292', '174360', '15728...","{'Home': {'Squad': ['129292', '174360', '15728...","{'Home': {'Squad': ['129292', '174360', '15728...",NaN,NaN
2,"{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...","{'Home': {'Squad': ['293428', '165198', '59870...",NaN,NaN
3,"{'Home': {'Squad': ['427979', '186564', '16068...","{'Home': {'Squad': ['427979', '186564', '16068...","{'Home': {'Squad': ['427979', '186564', '16068...","{'Home': {'Squad': ['427979', '186564', '16068...","{'Home': {'Squad': ['427979', '186564', '16068...",NaN,NaN,NaN,NaN
4,"{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...","{'Home': {'Squad': ['383559', '180083', '18078...",NaN
5,"{'Home': {'Squad': ['459582', '291542', '17241...","{'Home': {'Squad': ['459582', '172411', '61006...","{'Home': {'Squad': ['459582', '172411', '61006...","{'Home': {'Squad': ['459582', '172411', '61006...","{'Home': {'Squad': ['459582', '172411', '61006...","{'Home': {'Squad': ['459582', '172411', '61006...",NaN,NaN,NaN
6,"{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...","{'Home': {'Squad': ['511533', '307840', '54090...",NaN,NaN,NaN
7,"{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...","{'Home': {'Squad': ['154179', '316645', '18484...",NaN,NaN
8,"{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '331842', '30898...","{'Home': {'Squad': ['177770', '308980', '33783...",NaN,NaN,NaN
9,"{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...","{'Home': {'Squad': ['166276', '161301', '54157...",NaN,NaN


Teste inicial utilizando um jogo do brasileirão Série A 2014. Pegamos o primeiro jogo do Fluminense x Figueirense para criarmos a formatação das colunas e extração de informações da súmula.

In [102]:
players_teams = df_transposto.iloc[24, 3]
team_changes = df_transposto.iloc[24, 5]

start_players = df_squad.iloc[24,:]

# Criando DataFrame dos jogadores
new_df_players = pd.DataFrame(players_teams, columns=['player', 'team'])

def extract_id(player_string):
    # Ajuste para garantir que captura corretamente todas as variantes de ID
    match = re.search(r'T\(g\)?P(\d+)|RP(\d+)|TP(\d+)', player_string, re.IGNORECASE)
    if match:
        return next((m for m in match.groups() if m), None)
    return None

# Limpeza dos nomes dos jogadores
def clean_player_name(player):
    clean_name = re.sub(r'\s+T.*|\s+TP.*|\s+RP.*|\s+T(g).*', '', player)  # Remove texto após 'T' ou 'TP'
    return clean_name

new_df_players['player_id'] = new_df_players['player'].apply(extract_id)
new_df_players['player_name'] = new_df_players['player'].apply(clean_player_name)


def parse_team_changes(changes):
    pattern = re.compile(r'(\d{2}:\d{2}) (INT|\d+T)([\w\s]+/\w+) (\d+) - [^\d]+ (\d+) - [^\d]+')
    parsed_data = []
    for change in changes:

        match = pattern.search(change)
        if match:
            time, half, team, player_out_number, player_in_number = match.groups()
            team = team.strip()  # Limpeza para remover espaços extras
            parsed_data.append((time, half, team, player_out_number, player_in_number))
    return parsed_data


parsed_changes = parse_team_changes(team_changes)
#print(parsed_changes[0])


new_df_players['Minutes Played'] = 90
new_df_players['Minute Entered'] = 0  # Todos os jogadores começam com entrada no minuto 0
new_df_players['Minute Exited'] = 90  # Todos os jogadores começam com saída no minuto 90

def verifica_substring(substring, string):
    partes = substring.split()
    return all(parte in string for parte in partes)


def tratar_excecoes_nomes_times(team):
    # Dicionário de exceções: chave é o nome parcial, valor é o nome completo
    excecoes = {
        "Atlético/PR": "Athletico Paranaense/PR",
        "Atlético/MG": "Atlético Mineiro/MG"
        # Adicione outras exceções aqui conforme necessário
    }
    
    # Verificar se o time está nas exceções e substituí-lo
    return excecoes.get(team, team)


for time, half, team, player_out_number, player_in_number in parsed_changes:

    team = tratar_excecoes_nomes_times(team)
    minute = int(time.split(':')[0])
    team = team.replace('/', ' / ')  # Formatando o nome do time corretamente
    
    if '2T' in half:
        minute_entered = 45 + minute
        minute_exited = 45 + minute
    else:
        minute_entered = minute
        minute_exited = minute
    print(team)
    # Atualizar o jogador que entrou
    mask_in = (new_df_players['player_name'].apply(lambda x: re.match(r'^' + player_in_number + r'\D', x) is not None) & 
               new_df_players['team'].apply(lambda x: verifica_substring(team, x)))
    new_df_players.loc[mask_in, 'Minute Entered'] = minute_entered
    new_df_players.loc[mask_in, 'Minute Exited'] = 90  # O jogador que entra fica até o final do jogo

    # Atualizar o jogador que saiu
    mask_out = (new_df_players['player_name'].apply(lambda x: re.match(r'^' + player_out_number + r'\D', x) is not None) & 
                new_df_players['team'].apply(lambda x: verifica_substring(team, x)))
    new_df_players.loc[mask_out, 'Minute Exited'] = minute_exited

new_df_players['Minutes Played'] = new_df_players['Minute Exited'] - new_df_players['Minute Entered']


#setando Home ou Away
home_team = df_transposto.iloc[24, 0]
new_df_players
new_df_players['status'] = np.where(new_df_players['team'] == home_team, 'Home', 'Away')

new_df_players = new_df_players.drop(columns='player')
# Visualizando o novo DataFrame
new_df_players

Ponte Preta / SP
Atlético Mineiro / MG
Atlético Mineiro / MG
Atlético Mineiro / MG
Ponte Preta / SP
Ponte Preta / SP


,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status
0,Atlético Mineiro / MG,156659,1Victor Victor Leandro Bagy,90,0,90,Home
1,Atlético Mineiro / MG,176390,2Marcos Rocha Marcos Luis Rocha Aquino,90,0,90,Home
2,Atlético Mineiro / MG,181262,5Rafael Car ... Rafael de Souza Pereira,90,0,90,Home
3,Atlético Mineiro / MG,162248,6Fabio Santos Fabio Santos Romeu,90,0,90,Home
4,Atlético Mineiro / MG,152954,7Robinho Robson de Souza,16,74,90,Home
5,Atlético Mineiro / MG,159607,8Elias Elias Mendes,90,0,90,Home
6,Atlético Mineiro / MG,148994,9Fred Frederico Chaves Guedes,90,0,90,Home
7,Atlético Mineiro / MG,540839,10Cazares Juan Ramon Cazares S ...,32,58,90,Home
8,Atlético Mineiro / MG,172685,21Adilson Adilson Warken,38,52,90,Home
9,Atlético Mineiro / MG,169374,26Felipe Felipe Augusto Santana,90,0,90,Home


In [100]:
def tratar_excecoes_nomes_times_2(team):
    # Dicionário de exceções: chave é o nome parcial, valor é o nome completo
    excecoes = {
        "Athletico Paranaense/PR": "Atlético/PR",
        "Atlético Mineiro/MG": "Atlético/MG"
        # Adicione outras exceções aqui conforme necessário
    }
    
    # Verificar se o time está nas exceções e substituí-lo
    return excecoes.get(team, team)

def parse_goals(goals, home_team):
    pattern = re.compile(r'(\+?\d+)(:00)? (\d+T)([^\s]+)\s+([^\d]+?)\s+(.*)')

    
    parsed_goals = []
    for goal in goals:

        match = pattern.search(goal)

        if match:
            if '+' in (match.group(1)):
                minute = 45
            else:
                minute = int(match.group(1))

            half = match.group(3)

            scorer_info = match.group(4)
            print(match.group(5))
            player_name = match.group(5).strip()

            time_casa = df_transposto.iloc[14, 0]
            time_visitante = df_transposto.iloc[14, 1]


            time_casa = time_casa.replace(' / ', '/')
            time_visitante = time_visitante.replace(' / ', '/')

            time_casa = tratar_excecoes_nomes_times_2(time_casa)

            if time_casa in  match.group(6):
                team = time_casa
            else: 
                team = time_visitante

            

            team = tratar_excecoes_nomes_times(team)
            team = team.replace('/', ' / ')
            print(home_team, team)
            # Convertendo para minuto total do jogo
            if '2T' in half and minute != 45:  # Ajusta para o tempo total no caso do segundo tempo
                minute += 45
            
            # Verificar se é um gol contra (CT)
            if 'CT' in scorer_info:
                if team == home_team:
                    team_status = 'Away'
                else:
                    team_status = 'Home'
            else:

                if team == home_team:
                    team_status = 'Home'
                else:
                    team_status = 'Away'
            
            parsed_goals.append((minute, team_status))
    return parsed_goals

home_team = df_transposto.iloc[14, 0]
parsed_goals = parse_goals(df_transposto.iloc[14,4], home_team)

parsed_goals

de
Atlético Mineiro / MG Fluminense / RJ
Costa
Atlético Mineiro / MG Atlético Mineiro / MG
Henrique
Atlético Mineiro / MG Fluminense / RJ


[(38, 'Away'), (40, 'Home'), (36, 'Away')]

In [71]:
df_transposto.iloc[25,4]

['24:00 1T11NRFederico Andres Mancuello Flamengo/RJ',
 '10:00 2T44NRThiago Heleno Henrique Ferreira Atlético/PR']

In [72]:
df_transposto.iloc[8,6]

['41:00 1T5Rodrigo Vasconcelos Oliveira Sport/PE',
 '43:00 1T27Samuel Xavier Brito Sport/PE']

In [73]:

new_df_players['Goals For'] = 0

new_df_players['Goals Against'] = 0

for minute, team in parsed_goals:
    # Jogadores em campo no momento do gol
    mask = (new_df_players['Minute Entered'] <= minute) & (new_df_players['Minute Exited'] >= minute)
    # Incrementar gols a favor
    mask_for = mask & (new_df_players['status'] == team)
    new_df_players.loc[mask_for, 'Goals For'] += 1
    
    # Incrementar gols contra
    mask_against = mask & (new_df_players['status'] != team)
    new_df_players.loc[mask_against, 'Goals Against'] += 1

# Exibindo o DataFrame final para verificar os resultados
new_df_players

,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status,Goals For,Goals Against
0,Atlético Mineiro / MG,156659,1Victor Victor Leandro Bagy,90,0,90,Home,1,1
1,Atlético Mineiro / MG,176390,2Marcos Rocha Marcos Luis Rocha Aquino,90,0,90,Home,1,1
2,Atlético Mineiro / MG,181262,5Rafael Car ... Rafael de Souza Pereira,45,45,90,Home,1,0
3,Atlético Mineiro / MG,162248,6Fabio Santos Fabio Santos Romeu,90,0,90,Home,1,1
4,Atlético Mineiro / MG,159607,8Elias Elias Mendes,90,0,90,Home,1,1
5,Atlético Mineiro / MG,148994,9Fred Frederico Chaves Guedes,90,0,90,Home,1,1
6,Atlético Mineiro / MG,540839,10Cazares Juan Ramon Cazares S ...,90,0,90,Home,1,1
7,Atlético Mineiro / MG,554728,11Otero Romulo Otero Vasquez,30,60,90,Home,0,0
8,Atlético Mineiro / MG,172685,21Adilson Adilson Warken,90,0,90,Home,1,1
9,Atlético Mineiro / MG,169374,26Felipe Felipe Augusto Santana,90,0,90,Home,1,1


In [74]:

def collect_unique_ids(data_folder, file_names, years):
    unique_ids = set()

    def process_dataframe(df):
        for column in df.columns:
            for cell in df[column]:
                if isinstance(cell, dict):  
                    for key in ['Home', 'Away']:
                        team_info = cell.get(key, {})
                        # Adicionar os IDs do 'Squad'
                        squad_ids = team_info.get('Squad', [])
                        unique_ids.update(squad_ids)


    for file_name in file_names:
        for year in years:
            file_path = os.path.join(data_folder, f'{file_name}_{year}_squads.json')
            if os.path.exists(file_path):
                df = pd.read_json(file_path)
                df_ = df.iloc[:,7]
                df = pd.DataFrame(df_)
                process_dataframe(df)
            else:
                print(f'Arquivo não encontrado: {file_path}')

    return list(unique_ids)

data_folder = '../Data/'
file_names = ['Serie_A']
years = range(2017, 2018)

unique_player_ids = collect_unique_ids(data_folder, file_names, years)

print(f'Total de IDs únicos coletados: {len(unique_player_ids)}')

Total de IDs únicos coletados: 28


In [75]:

df_players_final = new_df_players[new_df_players['player_id'].isin(unique_player_ids)]
df_players_final

,team,player_id,player_name,Minutes Played,Minute Entered,Minute Exited,status,Goals For,Goals Against
